# 範例 : (Kaggle)房價預測精簡版 
https://www.kaggle.com/c/house-prices-advanced-regression-techniques
----
***
- 以下是房價預測的精簡版範例
- 使用最小量的特徵工程以及線性回歸模型做預測, 最後輸出可以在Kaggle提交的預測檔

## 補充
* [了解其房價預測的資料資訊](https://www.kaggle.com/c/house-prices-advanced-regression-techniques/data)
* log1p(x) 和 expm1(x) 介紹
  * [log1p(x) 和 expm1(x) 函数的实现](https://blog.csdn.net/liyuanbhu/article/details/8544644)
    * log1p(x) := log(1+x)
    * expm1(x) := exp(x) - 1
    * 主要應該是當 x 很小時，因為有效位數的關係，我們得到的結果 log(1+x) 常為 0 ，而非真正很小的值。故以log1p(x)來取得實際的值。
  * [np.log1p( ) 函数的应用](https://blog.csdn.net/u012193416/article/details/83211016)
    * 数据预处理时首先可以对偏度比较大的数据用 log1p() 进行转化，使其更加服从高斯分布，此步处理可能会使我们后续的分类结果得到一个好的结果。
    * log1p() 的使用就像是一个数据压缩到了一个区间，与数据的标准类似。其逆运算就是expm1的函数
    * 由于使用的log1p（）对数据进行了压缩，最后需要将预测出的平滑数据进行一个还原，而还原过程就是 log1p() 的逆运算 expm1().
* MinMaxScaler()
  * [sklearn.preprocessing.MinMaxScaler](https://scikit-learn.org/stable/modules/generated/sklearn.preprocessing.MinMaxScaler.html)
    * Transforms features by scaling each feature to a given range.
  * [关于使用sklearn进行数据预处理 —— 归一化/标准化/正则化](https://www.cnblogs.com/chaosimple/p/4153167.html)

In [ ]:
# 載入套件
import pandas as pd
import numpy as np
from sklearn.preprocessing import LabelEncoder, MinMaxScaler

#讀取訓練與測試資料
data_path = 'data/'
df_train = pd.read_csv(data_path + 'house_train.csv.gz')
df_test = pd.read_csv(data_path + 'house_test.csv.gz')
print(df_train.shape)

In [ ]:
# 觀察 np.log1p() 與 np.expm1() 的轉換過程
print(df_train['SalePrice'].head())
print(np.log1p(df_train['SalePrice'].head()))
tmp = df_train['SalePrice'].head(1)
print(tmp)
print(np.log1p(tmp))
print(np.expm1(np.log1p(tmp)))

In [ ]:
# 訓練資料需要 train_X, train_Y / 預測輸出需要 ids(識別每個預測值), test_X
# 在此先抽離出 train_Y 與 ids, 而先將 train_X, test_X 該有的資料合併成 df, 先作特徵工程

# SalePrice - the property's sale price in dollars. This is the target variable that you're trying to predict.
train_Y = np.log1p(df_train['SalePrice'])
ids = df_test['Id']
df_train = df_train.drop(['Id', 'SalePrice'] , axis=1)
df_test = df_test.drop(['Id'] , axis=1)
df = pd.concat([df_train,df_test])
df.head()

In [ ]:
# 特徵工程-簡化版 : 全部空值先補-1, 所有類別欄位先做 LabelEncoder, 然後再與數字欄位做MinMaxScaler
# 這區塊的細節會在後續的課程做解說
LEncoder = LabelEncoder()
MMEncoder = MinMaxScaler()
for c in df.columns:
    df[c] = df[c].fillna(-1)
    if df[c].dtype == 'object':
        df[c] = LEncoder.fit_transform(list(df[c].values))
    df[c] = MMEncoder.fit_transform(df[c].values.reshape(-1, 1))
df.head()

In [ ]:
# 將前述轉換完畢資料 df , 重新切成 train_X, test_X
train_num = train_Y.shape[0]
train_X = df[:train_num]
test_X = df[train_num:]

# 使用線性迴歸模型
# train_X, train_Y 訓練模型, 並對 test_X 做出預測結果 pred
from sklearn.linear_model import LinearRegression
estimator = LinearRegression()
estimator.fit(train_X, train_Y)
pred = estimator.predict(test_X)

In [ ]:
# 將輸出結果 pred 與前面留下的 ID(ids) 合併, 輸出成檔案
# 可以下載並點開 house_baseline.csv 查看結果, 以便了解預測結果的輸出格式
# 本範例所與作業所輸出的 csv 檔, 均可用於本題的 Kaggle 答案上傳, 可以試著上傳來熟悉 Kaggle 的介面操作
pred = np.expm1(pred)
sub = pd.DataFrame({'Id': ids, 'SalePrice': pred})
sub.to_csv('house_baseline.csv', index=False) 

# 作業1
* 下列A~E五個程式區塊中，哪一塊是特徵工程?

# 作業2
* 對照程式區塊 B 與 C 的結果，請問那些欄位屬於"類別型欄位"? (回答欄位英文名稱即可) 

# 作業3
* 續上題，請問哪個欄位是"目標值"?